In [ ]:
!pip install ipypb

In [1]:
import os
import pandas as pd
import numpy as np
from ipypb import ipb
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from math import ceil 

In [ ]:
test_path = ''
rand_path = ''
ecg_path = ''
save_path = ''

In [ ]:
# import data table including ecg_id
df_test = pd.read_csv(test_path)
df_test.head()

In [ ]:
# extract 100 ecg randomly
np.random.seed(seed=0)
rand_int = np.random.randint(0, len(df_test), size=100)
df_100 = df_test.iloc[rand_int]
ecg_100 = df_100['ecg_id'].to_list()
print(len(ecg_100))

In [ ]:
# save the metadata of 100 ecg extracted randomly
df_100.reset_index(inplace=True)
df_100.to_csv(rand_path+'testset_100.csv', index=True)

ecg-plot modifying
- https://pypi.org/project/ecg-plot/

In [ ]:
ecg_folder = ecg_path
lead_form = ['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'III', 'aVR', 'aVL', 'aVF']

def signal_set(ecg_id):
    df = pd.read_csv(ecg_folder+ecg_id+'.csv')
    df = df[lead_form]
    df_norm = df.applymap(lambda x: x/1000)
    ecg_data = df_norm.to_numpy()
    leads = {
        # col 1
        'I': ecg_data[0:1250, 0],
        'II': ecg_data[0:1250, 1],
        'III': ecg_data[0:1250, 8],
        'II ': ecg_data[0:1250, 1],
        # col 2
        'aVR': ecg_data[1250:2500, 9],
        'aVL': ecg_data[1250:2500, 10],
        'aVF': ecg_data[1250:2500, 11],
        '': ecg_data[1250:2500, 1],
        # col 3
        'V1': ecg_data[2500:3750, 2],
        'V2': ecg_data[2500:3750, 3],
        'V3': ecg_data[2500:3750, 4],
        ' ': ecg_data[2500:3750, 1],
        # col 4
        'V4': ecg_data[3750:5000, 5],
        'V5': ecg_data[3750:5000, 6],
        'V6': ecg_data[3750:5000, 7],
        '  ': ecg_data[3750:5000, 1],
    }
    ecg_leads = np.array(list(leads.values()))
    return ecg_leads

def _ax_plot(ax, x, y, secs=10, lwidth=0.5, amplitude_ecg = 1.8, time_ticks =0.2):
    #ax.set_xticks(np.arange(0,11,time_ticks))    
    #ax.set_yticks(np.arange(-ceil(amplitude_ecg),ceil(amplitude_ecg),1.0))

    #ax.set_yticklabels([])
    #ax.set_xticklabels([])

    ax.minorticks_on()
    
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))

    #ax.set_ylim(-amplitude_ecg, amplitude_ecg)
    #ax.set_xlim(0, secs)

    ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
    ax.grid(which='minor', linestyle='-', linewidth='0.5', color=(1, 0.7, 0.7))

    ax.plot(x,y, linewidth=lwidth)


lead_index = ['I', 'II', 'III', 'II ', 'aVR', 'aVL', 'aVF', '', 'V1', 'V2', 'V3', ' ', 'V4', 'V5', 'V6', '  ']

def plot(
        ecg, title,
        sample_rate    = 500,  
        lead_index     = lead_index, 
        lead_order     = None,
        style          = None,
        columns        = 4,
        row_height     = 6,
        show_lead_name = True,
        show_grid      = True,
        show_separate_line  = None,
        ):
    """Plot multi lead ECG chart.
    # Arguments
        ecg        : m x n ECG signal data, which m is number of leads and n is length of signal.
        sample_rate: Sample rate of the signal.
        title      : Title which will be shown on top off chart
        lead_index : Lead name array in the same order of ecg, will be shown on 
            left of signal plot, defaults to ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
        lead_order : Lead display order 
        columns    : display columns, defaults to 2
        style      : display style, defaults to None, can be 'bw' which means black white
        row_height :   how many grid should a lead signal have,
        show_lead_name : show lead name
        show_grid      : show grid
        show_separate_line  : show separate line
    """

    if not lead_order:
        lead_order = list(range(0,len(ecg)))
    secs  = len(ecg[0])/sample_rate
    leads = len(lead_order)
    rows  = int(ceil(leads/columns))
    # display_factor = 2.5
    display_factor = 1
    line_width = 0.5
    fig, ax = plt.subplots(figsize=(secs*columns * display_factor, rows * row_height / 5 * display_factor))
    display_factor = display_factor ** 0.5
    fig.subplots_adjust(
        hspace = 0, 
        wspace = 0,
        left   = 0,  # the left side of the subplots of the figure
        right  = 1,  # the right side of the subplots of the figure
        bottom = 0,  # the bottom of the subplots of the figure
        top    = 1
        )

    fig.suptitle(title)

    x_min = 0
    x_max = columns*secs
    y_min = row_height/4 - (rows/2)*row_height
    y_max = row_height/4

    if (style == 'bw'):
        color_major = (0.4,0.4,0.4)
        color_minor = (0.75, 0.75, 0.75)
        color_line  = (0,0,0)
    else:
        color_major = (1,0,0)
        color_minor = (1, 0.7, 0.7)
        color_line  = (0,0,0.7)

    if(show_grid):
        ax.set_xticks(np.arange(x_min,x_max,0.2))    
        ax.set_yticks(np.arange(y_min,y_max,0.5))

        ax.minorticks_on()
        
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))

        ax.grid(which='major', linestyle='-', linewidth=0.5 * display_factor, color=color_major)
        ax.grid(which='minor', linestyle='-', linewidth=0.5 * display_factor, color=color_minor)

    ax.set_ylim(y_min,y_max)
    ax.set_xlim(x_min,x_max)

    ax.set_yticklabels([])
    ax.set_xticklabels([])



    for c in range(0, columns):
        for i in range(0, rows):
            if (c * rows + i < leads):
                y_offset = -(row_height/2) * ceil(i%rows)
                # if (y_offset < -5):
                #     y_offset = y_offset + 0.25

                x_offset = 0
                if(c > 0):
                    x_offset = secs * c
                    if(show_separate_line):
                        ax.plot([x_offset, x_offset], [ecg[t_lead][0] + y_offset - 0.3, ecg[t_lead][0] + y_offset + 0.3], linewidth=line_width * display_factor, color=color_line)

         
                t_lead = lead_order[c * rows + i]
         
                step = 1.0/sample_rate
                if(show_lead_name):
                    ax.text(x_offset + 0.07, y_offset - 0.5, lead_index[t_lead], fontsize=9 * display_factor)
                ax.plot(
                    np.arange(0, len(ecg[t_lead])*step, step) + x_offset, 
                    ecg[t_lead] + y_offset,
                    linewidth=line_width * display_factor, 
                    color=color_line
                    )


def plot_save(
        ecg, title,
        fname,
        sample_rate    = 500, 
        lead_index     = lead_index, 
        lead_order     = None,
        style          = None,
        columns        = 4,
        row_height     = 6,
        show_lead_name = True,
        show_grid      = True,
        show_separate_line  = None,
        ):


    if not lead_order:
        lead_order = list(range(0,len(ecg)))
    secs  = len(ecg[0])/sample_rate
    leads = len(lead_order)
    rows  = int(ceil(leads/columns))
    # display_factor = 2.5
    display_factor = 1
    line_width = 0.5
    fig, ax = plt.subplots(figsize=(secs*columns * display_factor, rows * row_height / 5 * display_factor))
    display_factor = display_factor ** 0.5
    fig.subplots_adjust(
        hspace = 0, 
        wspace = 0,
        left   = 0,  # the left side of the subplots of the figure
        right  = 1,  # the right side of the subplots of the figure
        bottom = 0,  # the bottom of the subplots of the figure
        top    = 1
        )

    fig.suptitle(title)

    x_min = 0
    x_max = columns*secs
    y_min = row_height/4 - (rows/2)*row_height
    y_max = row_height/4

    if (style == 'bw'):
        color_major = (0.4,0.4,0.4)
        color_minor = (0.75, 0.75, 0.75)
        color_line  = (0,0,0)
    else:
        color_major = (1,0,0)
        color_minor = (1, 0.7, 0.7)
        color_line  = (0,0,0.7)

    if(show_grid):
        ax.set_xticks(np.arange(x_min,x_max,0.2))    
        ax.set_yticks(np.arange(y_min,y_max,0.5))

        ax.minorticks_on()
        
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))

        ax.grid(which='major', linestyle='-', linewidth=0.5 * display_factor, color=color_major)
        ax.grid(which='minor', linestyle='-', linewidth=0.5 * display_factor, color=color_minor)

    ax.set_ylim(y_min,y_max)
    ax.set_xlim(x_min,x_max)

    ax.set_yticklabels([])
    ax.set_xticklabels([])



    for c in range(0, columns):
        for i in range(0, rows):
            if (c * rows + i < leads):
                y_offset = -(row_height/2) * ceil(i%rows)
                # if (y_offset < -5):
                #     y_offset = y_offset + 0.25

                x_offset = 0
                if(c > 0):
                    x_offset = secs * c
                    if(show_separate_line):
                        ax.plot([x_offset, x_offset], [ecg[t_lead][0] + y_offset - 0.3, ecg[t_lead][0] + y_offset + 0.3], linewidth=line_width * display_factor, color=color_line)

         
                t_lead = lead_order[c * rows + i]
         
                step = 1.0/sample_rate
                if(show_lead_name):
                    ax.text(x_offset + 0.07, y_offset - 0.5, lead_index[t_lead], fontsize=9 * display_factor)
                ax.plot(
                    np.arange(0, len(ecg[t_lead])*step, step) + x_offset, 
                    ecg[t_lead] + y_offset,
                    linewidth=line_width * display_factor, 
                    color=color_line
                    )
    plt.savefig(save_path+'_'+fname+'.png', format='png', dpi=300)
    

def ecg_view(ecg_id, ecg_num):
    ecg_signal = signal_set(ecg_id)
    ecg_num = str(ecg_num)
    plot(ecg_signal, ecg_num)

def ecg_save(ecg_id, ecg_num, fname):
    ecg_signal = signal_set(ecg_id)
    ecg_num = str(ecg_num)
    plot_save(ecg_signal, ecg_num, fname)

In [ ]:
# view the ecg plot
ecg_view(ecg_100[0], 0)

In [ ]:
# save the ecg plot
for i in ipb(range(len(ecg_100))):
    ecg = ecg_100[i]
    ecg_save(ecg, i, str(i))